Final Project Submission

Student name: Mark Ehler <br>
Student pace: part time <br>
Scheduled project review date/time: -- --  2019 -- MDT US <br>
Instructor name: Jeff Herman <br>
Blog post URL:

<h4> RNN For Sentiment Analysis </h4>

Intro for final model...

This module uses the <a href=" https://www.kaggle.com/zynicide/wine-reviewsdata ">Wine Reviews </a> dataset availible on Kaggle to determine sentiment analysis of the wine after reading only a short review.  The goal being to deveolop an RNN that can organize text data. Results will be ran against a straight-forward logistic regression model test the strength of the sentiment analysis model.


In [2]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Input, Embedding, Flatten, Dense, Dropout, Activation, Bidirectional, LSTM, GlobalMaxPool1D #different layers we might use
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical, Sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import  sequence
from keras.callbacks import History 

C:\Users\Mark\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [3]:
%store -r 

reviews = pd.read_csv('_review_data.csv') # all cleaned data
stop_removed_text = stop_removed_text # preprocessed but unsorted
trainDataVecs = trainDataVecs # presorted processed tokens
testDataVecs = testDataVecs # presorted processed tokens
print(reviews.columns)


Index(['Unnamed: 0', 'description', 'two_outcomes'], dtype='object')


In [4]:

total_vocabulary = set(word for line in stop_removed_text for word in line)

len(total_vocabulary)

30241

In [5]:
glove_dir = r'C:\Users\Mark\Documents\DataSci\glove_dir\glove.6B.50d.txt' #r prefix converts from a normal string to a raw string
glove = {}
with open(glove_dir, 'rb') as f:
    for line in f:
        parts = line.split()
        word = parts[0].decode('utf-8')
        if word in total_vocabulary:
            vector = np.array(parts[1:], dtype=np.float32)
            glove[word] = vector
print(f'Found {len(glove)} word vectors.')

Found 24127 word vectors.


<h3> Tokenize the Text </h3>

In [6]:
# Adapted from Deep learning RNN Chapter
maxlen = 40 # no reviews are longer than 40 words
max_words = 10000 # do I use this # for the embedding layer my data.shape[0] which is 129907 for the entire set 
def keras_enc(samples, maxlen=maxlen):
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(samples)

    sample_sequences = tokenizer.texts_to_sequences(samples)

    word_index = tokenizer.word_index
    print(f'found {len(word_index)} unique tokens')
    
    samples_transformed = sequence.pad_sequences(sample_sequences, maxlen=maxlen)
    
    return samples_transformed, word_index

In [7]:
desc_t, word_index = keras_enc(stop_removed_text)

found 30241 unique tokens


In [8]:
glove_dir = r'C:\Users\Mark\Documents\DataSci\glove_dir\glove.6B.50d.txt' #r prefix converts from a normal string to a raw string
embedding_index = {}
with open(glove_dir, 'rb') as f:
    for line in f:
        parts = line.split()
        word = parts[0].decode('utf-8')
        if word in word_index.keys():
            vector = np.array(parts[1:], dtype=np.float32)
            embedding_index[word] = vector
print(f'Found {len(embedding_index)} word vectors.')
print(f'{round(( len(embedding_index)/len(total_vocabulary) *100),2)}% word coverage')

Found 24127 word vectors.
79.78% word coverage


In [9]:
embedding_dim = 50 # GloVe seems to use arrays in shape (50,)
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [86]:
embedding_matrix[1]

array([-0.1145    ,  0.75404   , -1.64320004, -0.61037999,  0.60351998,
       -0.56396002, -1.00689995, -0.44103   ,  0.61255997,  1.18120003,
        0.18128   ,  0.30032   ,  1.18169999, -0.62548   ,  1.21560001,
       -0.30737999,  0.54095   ,  0.53758001, -0.026086  , -1.73870003,
        0.46533   , -0.62835002,  0.50936002,  1.11919999, -0.74747002,
       -0.57528001, -0.92030001,  0.98611999,  0.29107001,  0.60207999,
        1.97029996, -0.27461001, -0.34920999,  0.44141001,  0.64402002,
       -0.32352999, -1.45410001,  1.14719999,  0.86874998, -0.074512  ,
        0.85632002,  0.59341002,  0.4655    , -0.0387    ,  0.26462999,
        0.94151002, -0.27335   , -0.085403  ,  0.12693   , -0.23861   ])

In [87]:
input_shape = desc_t.shape
X_nn = desc_t
input_shape

(129907, 40)

In [88]:
# shape of labels. this variable will be used to fit model
labels = reviews['two_outcomes']
labels.shape

(129907,)

In [94]:
#shape of data before tokenization
reviews['description'].shape

(129907,)

In [93]:
# shape of data after tokenization
X_nn.shape

(129907, 40)

In [95]:
# data was used in this vector to build a random forest model
trainDataVecs.shape

(90935, 50)

In [101]:
test_model = Sequential()
test_model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
test_model.compile(optimizer='adam', loss = 'mse')
test_model.fit(X_nn, labels, epochs=2, batch_size=128,
                    validation_split=0.1, callbacks=callbacks)

ValueError: Error when checking target: expected embedding_7 to have 3 dimensions, but got array with shape (129907, 1)

In [96]:
lstm_model = Sequential()
lstm_model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
lstm_model.add(LSTM(50, return_sequences=True))
lstm_model.add(GlobalMaxPool1D())
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(50, activation='relu'))
lstm_model.add(Dropout(0.25))
lstm_model.add(Dense(1, activation='softmax'))
lstm_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 40, 50)            500000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 40, 50)            20200     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 50)                0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_8 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 51        
Total para

In [12]:
# lstm_model.layers[0].set_weights([embedding_matrix])
# model.layers[0].trainable = False

In [98]:
lstm_model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc'])

In [99]:
checkpoints_path = 'wine_reviews.best.hdf5'
checkpoint = ModelCheckpoint(checkpoints_path, monitor='val_loss', 
                             verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=10)
history = History()
callbacks = [checkpoint, early_stopping, history]

In [10]:
%%time

history = lstm_model.fit(trainDataVecs, train_reviews['two_outcomes'], epochs=2, batch_size=128,
                    validation_split=0.1, callbacks=callbacks)

NameError: name 'lstm_model' is not defined

In [56]:
history.history.keys()

NameError: name 'history' is not defined

In [52]:
def plot_results(hist):
    acc = hist.history['acc']
    val_acc = hist.history['val_acc']
    loss = hist.history['loss']
    val_loss = hist.history['val_loss']
    
    epochs = range(1,len(acc) + 1)
    
    plt.plot(epochs, acc, 'bo', label='Training Accuracy')
    plt.plot(epochs, val_acc, 'b', label = 'Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend(loc='best')
    
    plt.plot(epochs, loss, 'bo', label='Training Loss', color='orange')
    plt.plot(epochs, val_loss, 'b', label='Validation Loss', color='orange')
    plt.title('Training and Validation Loss')
    plt.legend()
    
    plt.show();
    
plot_results(history)

NameError: name 'history' is not defined